# offline habitat map generation using the openEO generated CatBoost models and feature data cubes
In this notebook we apply the generated CatBoost models on the openEO generated feature data cubes to run the hierarchical EUNIS habitat mapping. Output are habitat occurrence probability layers for each EUNIS habitat existing in the test sites.

In [1]:
import platform
import os
import pandas as pd
import json
import glob
import rioxarray
from catboost import CatBoostClassifier
import numpy as np
import xarray as xr
from tqdm.notebook import tqdm

In [2]:
# get input and output root folders
if platform.system() == 'Windows':
    path_root_models = os.path.normpath(r'\\netapp03.vgt.vito.be\habitat\slovakia\openEO_tests\alpha-1\2_model_training')
    path_root_cubes = os.path.normpath(r'\\netapp03.vgt.vito.be\habitat\slovakia\openEO_tests\alpha-1\3_datacubes_test-sides')
    path_root_out = os.path.normpath(r'\\netapp03.vgt.vito.be\habitat\slovakia\openEO_tests\alpha-1\4_inference-offline')
    path_encoder = os.path.normpath(r'\\netapp03.vgt.vito.be\habitat\LUT\EUNIS2007_LUT.csv')
else:
    path_root_models = os.path.normpath(r'/data/habitat/slovakia/openEO_tests/alpha-1/2_model_training')
    path_root_cubes = os.path.normpath(r'/data/habitat/slovakia/openEO_tests/alpha-1/3_datacubes_test-sides')
    path_root_out = os.path.normpath(r'/data/habitat/slovakia/openEO_tests/alpha-1/4_inference-offline')
    path_encoder = os.path.normpath(r'/data/habitat/LUT/EUNIS2007_LUT.csv')

### get overview of models and input cubes

In [3]:
# Scan path_root_cubes for .tif files and add Paths to a pandas DataFrame
tif_files = glob.glob(os.path.join(path_root_cubes, '**', '*.tif'), recursive=True)
df_tif_paths = pd.DataFrame(tif_files, columns=['file_path'])
df_tif_paths['basename'] = df_tif_paths['file_path'].apply(lambda x: os.path.basename(x))
df_tif_paths = df_tif_paths.join(df_tif_paths['basename'].str.split('_', expand=True).rename(columns={0: 'version', 1: 'site', 2: 'type', 3: 'year', 4: 'tile'}))
df_tif_paths['year'] = pd.to_datetime(df_tif_paths['year'], format='year%Y')

In [4]:
df_tif_paths.head()

,file_path,basename,version,site,type,year,tile
0,\\netapp03.vgt.vito.be\habitat\slovakia\openEO...,alpha-1_test-sites_feature-cube_year2021_E468N...,alpha-1,test-sites,feature-cube,2021-01-01,E468N306.tif
1,\\netapp03.vgt.vito.be\habitat\slovakia\openEO...,alpha-1_test-sites_feature-cube_year2021_E468N...,alpha-1,test-sites,feature-cube,2021-01-01,E468N308.tif
2,\\netapp03.vgt.vito.be\habitat\slovakia\openEO...,alpha-1_test-sites_feature-cube_year2021_E470N...,alpha-1,test-sites,feature-cube,2021-01-01,E470N306.tif
3,\\netapp03.vgt.vito.be\habitat\slovakia\openEO...,alpha-1_test-sites_feature-cube_year2021_E470N...,alpha-1,test-sites,feature-cube,2021-01-01,E470N308.tif
4,\\netapp03.vgt.vito.be\habitat\slovakia\openEO...,alpha-1_test-sites_feature-cube_year2021_E472N...,alpha-1,test-sites,feature-cube,2021-01-01,E472N306.tif


In [5]:
# Scan path_root_models for a .json file and load it into a dictionary
json_files = glob.glob(os.path.join(path_root_models, '*.json'))
if json_files:
    with open(json_files[0], 'r') as f:
        model_dict = json.load(f)

In [6]:
# transfer the model information into a hierarchical Dataframe
# Function to create a DataFrame with MultiIndex from model_dict
def create_hierarchical_dataframe(model_dict):
    classification_levels = []
    habitat_classes = []
    model_paths = []

    for key, value in model_dict.items():
        parts = key.split('_')
        if len(parts) >= 2:
            classification_level, habitat_class = parts[0], parts[1]
            classification_levels.append(classification_level)
            habitat_classes.append(habitat_class)
            model_paths.append(value)

    index = pd.MultiIndex.from_arrays([classification_levels, habitat_classes], names=['classification_level', 'habitat_class'])
    df = pd.DataFrame({'model_path': model_paths}, index=index)

    return df

# Convert keys into a hierarchical DataFrame structure
hierarchical_model_df = create_hierarchical_dataframe(model_dict)

In [7]:
hierarchical_model_df.head()

model_path
classification_level habitat_class                                                   
Level1               class-0        /data/habitat/slovakia/openEO_tests/alpha-1/2_...
Level2               class-C        /data/habitat/slovakia/openEO_tests/alpha-1/2_...
                     class-D        /data/habitat/slovakia/openEO_tests/alpha-1/2_...
                     class-G        /data/habitat/slovakia/openEO_tests/alpha-1/2_...
                     class-F        /data/habitat/slovakia/openEO_tests/alpha-1/2_...

### set some functions for the processing

In [8]:
# habitat dict (encoder) -> raster value to habitat name
dhabitat = pd.read_csv(path_encoder, index_col=0).set_index('raster_value')['eunis_code'].to_dict()

In [22]:
# function to run the probability prediction
# NOTE: this processing is NOT USING block processing since we can still handle a 2000x2000 pixel cube with 253 bands in one go
def inference(data: xr.Dataset, row, label_encoder: dict) -> xr.Dataset:
    """
    Performs inference using a pre-trained CatBoost model on the provided dataset to predict
    habitat type probabilities, and then constructs an xarray.Dataset containing these
    probabilities. The prediction involves reshaping data to meet model requirements and
    converting predictions to a raster format with specified nodata values.

    :param data: An xarray.Dataset containing the input data for inference.
    :param row: A record containing model path and classification level details.
    :param label_encoder: A dictionary mapping class indices to habitat names.
    :return: An xarray.Dataset containing the predicted habitat type probabilities
        as data variables, with each variable having specified attributes and nodata values.
    """
    # get needed paths
    path_catboost_model = os.path.join(row.model_path, 'catboost_v1.cbm')
    path_predictors_model = os.path.join(row.model_path, 'predictors.json')

    # Replace 'data' with '\\netapp03.vgt.vito.be' if platform is Windows
    if platform.system() == 'Windows':
        path_catboost_model = path_catboost_model.replace('data', r'/netapp03.vgt.vito.be')
        path_predictors_model = path_predictors_model.replace('data', r'/netapp03.vgt.vito.be')

        # Ensure any '/' in the paths are converted to '\\' for Windows compatibility
        path_catboost_model = path_catboost_model.replace('/', '\\')
        path_predictors_model = path_predictors_model.replace('/', '\\')

    # Load the CatBoost model from the specified path
    catboost_model = CatBoostClassifier()
    catboost_model.load_model(path_catboost_model)

    # extract the feature names needed from json file
    with open(path_predictors_model, 'r') as f:
        predictors_data = json.load(f)

    # set the predictor names as feature names in the catboost model
    catboost_model.set_feature_names(predictors_data)

    # Filter the data by the variable names specified in predictors_data
    filtered_data = data[catboost_model.feature_names_]

    # info of original array shape
    len_y = filtered_data.sizes['y']
    len_x = filtered_data.sizes['x']
    len_features = len(filtered_data.data_vars)
    # info for result bands (probability layer for the single habitat types of the model)
    len_habitats = catboost_model.classes_.shape[0]

    ##### here comes the real work
    # Predict the probabilities with the CatBoost model on the filtered data
    # the Xaaray data is transfered to numpy array with shape [bands, y, x] and has to be reshaped to [y*x, bands] (known as [object_count, feature_count]])
    habitat_proba = catboost_model.predict_proba(filtered_data.to_array().values.reshape(len_features,len_y * len_x).T)

    # multiply with 100 to prepare for Byte format
    habitat_proba = habitat_proba * 100
    # now we have to reshape the habitat proba from [y*x, habitat_classes] to rasterio standard [bands, y, x]
    habitat_proba = habitat_proba.reshape(len_y, len_x, len_habitats).transpose(2, 0, 1).astype(np.uint8)

    ##### create the output xarray.Dataset
    # create the correct band names for the probability layers in the order of the catboost_model.classes_
    # EUNIS-level_model-name_habitat-name_habitat-raster-value
    # e.g. Level1_class-0_habitat-C-30000
    # e.g. Level2_class-C_habitat-C1-30100
    # e.g. Level3_class-C1_habitat-C1.1-30101
    blevel = row.classification_level
    bmodel = row.habitat_class
    bnames = [f'{blevel}_{bmodel}_habitat-{label_encoder[x]}-{x}' for x in catboost_model.classes_]

    # specify the nodata value
    nodata_val = 255

    # create the xarray.dataset
    habitat_proba_ds = xr.Dataset(
        {name: (['y', 'x'], habitat_proba[i]) for i, name in enumerate(bnames)},
        coords={
            'x': data.coords['x'],
            'y': data.coords['y']
        },
        attrs=data.attrs  # Copy over attributes from the original data
    )
    # set the crs
    habitat_proba_ds.rio.write_crs(data.rio.crs, inplace=True)
    # Set nodata value for each variable
    for var in habitat_proba_ds.data_vars:
        habitat_proba_ds[var].rio.write_nodata(nodata_val, inplace=True)
    # make sure all possible nan are replaced by new nodata value
    habitat_proba_ds = habitat_proba_ds.fillna(nodata_val)
    # del the openEO attribute
    del habitat_proba_ds.attrs['PROCESSING_SOFTWARE']

    # add some DataArray attributes
    for var in habitat_proba_ds.data_vars:
        habitat_proba_ds[var].encoding['habitat_class'] = var.split('_')[2].split('-')[1]
        habitat_proba_ds[var].encoding['class_raster_value'] = var.split('_')[2].split('-')[2]

    return habitat_proba_ds

## loop over all data cubes of the test sides and the models

In [23]:
#loop over all data cubes
for cube in tqdm(df_tif_paths.itertuples(), total=len(df_tif_paths), desc='processing data cubes'):
    # Load the test_path into a xarray DataArray with bands as variables
    data = rioxarray.open_rasterio(cube.file_path, band_as_variable=True, masked=True)
    # adapt the band name to the feature names extracted from the metadata
    data = data.rename({band:data[band].attrs["long_name"] for band in data})

    # init the list holding the Xarray Datasets for the final merge in the end
    lDatasets = []

    # loop over all models we have to run
    for model in tqdm(hierarchical_model_df.reset_index().itertuples(), total=len(hierarchical_model_df), desc='running models', leave=False):
        # run the prediction and xarray Dataset generation for the model and add to results
        lDatasets.append(inference(data, model, dhabitat))

    # merge all model Datasets into one big
    result = xr.merge(lDatasets)

    # output file name
    out_file_name = os.path.join(path_root_out, cube.basename.replace('feature-cube', 'habitat_probabilities'))

    # Save habitat_proba_ds as a Cloud Optimized GeoTIFF (COG)
    result.rio.to_raster(out_file_name, driver='COG')

    # free memory
    del data
    del lDatasets
    del result

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html